In [4]:
import numpy as np

print("请输入参评数目: ")
n=int(input())    #接收参评数目
print("请输入指标数目: ")
m=int(input())    #接收指标数目

#接收用户输入的类型矩阵,该矩阵指示了每个指标的类型(极大型,极小型等等)
print("请输入类型矩阵: 1:极大型,2:极小型,3:中间型,4:区间型")
kind=input().split(" ")   #将输入的字符串按空格分割,形成列表

#接收用户输入的矩阵并转化为numpy数组
print("请输入矩阵: ")
A=np.zeros(shape=(n,m))   #初始化一个全为0的矩阵
for i in range(n):
    A[i]=input().split(" ")   #按空格分割输入的字符串,并将结果存入矩阵的第i行
    A[i]=list(map(float,A[i]))   #将字符串转化为浮点数,并存入矩阵的第i行
print("输入矩阵为:\n{}".format(A))   #打印输入的矩阵A


#极小型指标转化为极大型指标的函数
def minTomax(maxx,x):
    x=list(x)     #将指标数组转化为列表
    ans=[[(maxx-e)] for e in x]   #计算每个值与最大值的差值,并将结果存入新的列表
    return np.array(ans)   #将列表转化为numpy数组

#中间型指标转化为极大型指标的函数
def midTomax(bestx,x):
    x=list(x)     #将指标数组转化为列表
    h=[abs(e-bestx) for e in x]   #计算每个值与最优值之间的绝对差值,并存入列表h
    M=max(h)   #计算h中的最大值
    if M==0:
        M=1     #如果最大值等于0,则将其替换为1
    ans=[[(1-e)/M] for e in h]   #计算每个值与最大值的差值除以2,并加上最大值的1/2,并将结果存入新的列表
    return np.array(ans)   #将列表转化为numpy数组

#区间型指标转化为极大型指标的函数
def intervalTomax(lowx,highx,x):
    x=list(x)     #将指标数组转化为列表
    M=max(lowx-min(x),max(x)-highx)   #计算区间型指标的最大值
    if M==0:
        M=1     #如果最大值等于0,则将其替换为1
    ans=[]
    for i in range(len(x)):
        if x[i]<lowx:
            ans.append([1-(lowx-x[i])/M])   #如果指标值小于下界,则将其转化为下界与最大值的差值除以最大值
        elif x[i]>highx:
            ans.append([1-(x[i]-highx)/M])   #如果指标值大于上界,则将其转化为上界与最大值的差值除以最大值
        else:
            ans.append([1])   #如果指标值在区间内,则将其转化为1
    return np.array(ans)   #将列表转化为numpy数组

#统一指标类型,将所有指标转化为极大型指标
X=np.zeros(shape=(n,1))   
for i in range(m):
    if kind[i]=="1":   #如果指标类型为极大型
        v=np.array(A[:,i])  
    elif kind[i]=="2":   #如果指标类型为极小型
        maxA=max(A[:,i])   #计算最大值
        v=minTomax(maxA,A[:,i])
    elif kind[i]=="3":   #如果指标类型为中间型
        print("类型三,请输入最优值: ")
        bestA=eval(input())   #接收最优值
        v=midTomax(bestA,A[:,i])
    elif kind[i]=="4":   #如果指标类型为区间型
        print("类型四,请输入区间[a,b]的下界a: ")
        lowA=eval(input())   #接收下界
        print("请输入区间[a,b]的上界b: ")
        highA=eval(input())   #接收上界
        v=intervalTomax(lowA,highA,A[:,i])
    if i==0:
        X=v.reshape(-1,1)   #如果是第一个指标,则将其赋值给X
    else:
        X=np.hstack([X,v.reshape(-1,1)])   #如果不是第一个指标,则将其添加到X的右侧
print("转换后的矩阵为:\n{}".format(X))   #打印转换后的矩阵X

#对统一指标后的矩阵X进行标准化处理:
X=X.astype('float')   #将X转化为浮点数
for j in range(m):
    X[:,j]=X[:,j]/np.sqrt(sum(X[:,j]**2))   #对每一列进行标准化处理
print("标准化后的矩阵为:\n{}".format(X))   #打印标准化后的矩阵X

#最大值最小值计算:
x_max=np.max(X,axis=0)   #计算每一列的最大值
x_min=np.min(X,axis=0)   #计算每一列的最小值
d_z=np.sqrt(np.sum(np.square((X-np.tile(x_max,(n,1)))),axis=1)) #计算与最优值之间的欧式距离d+
d_f=np.sqrt(np.sum(np.square((X-np.tile(x_min,(n,1)))),axis=1)) #计算与最差值之间的欧式距离d-
print('每个指标的最大值:',x_max)
print('每个指标的最小值:',x_min)
print('d+向量:',d_z)
print('d-向量:',d_f)

#计算每个参评的对象的得分排名
s=d_f/(d_z+d_f)   #计算每个参评对象的得分,接近1代表优,接近0代表劣
Score=100*s/sum(s)   #计算每个参评对象的百分制得分
for i in range(len(Score)):
    print(f"第{i+1}个标准化后百分制得分为:{Score[i]}")



请输入参评数目: 
请输入指标数目: 
请输入类型矩阵: 1:极大型,2:极小型,3:中间型,4:区间型
请输入矩阵: 
输入矩阵为:
[[  9.  10. 175. 120.]
 [  8.   7. 164.  80.]
 [  6.   3. 157.  90.]]
类型三,请输入最优值: 
类型四,请输入区间[a,b]的下界a: 
请输入区间[a,b]的上界b: 
转换后的矩阵为:
[[ 9.   0.  -0.9  0. ]
 [ 8.   3.   0.   0.5]
 [ 6.   7.  -0.7  1. ]]
标准化后的矩阵为:
[[ 0.66896473  0.         -0.78935222  0.        ]
 [ 0.59463532  0.3939193   0.          0.4472136 ]
 [ 0.44597649  0.91914503 -0.61394061  0.89442719]]
每个指标的最大值: [0.66896473 0.91914503 0.         0.89442719]
每个指标的最小值: [ 0.44597649  0.         -0.78935222  0.        ]
d+向量: [1.50595634 0.69382053 0.65318208]
d-向量: [0.22298824 1.00017438 1.29444846]
第1个标准化后百分制得分为:9.318738814052299
第2个标准化后百分制得分为:42.659903642138474
第3个标准化后百分制得分为:48.021357543809216


##### 新函数
- `split(" ")`将其字符串分割,返回列表

```python
kind="Hello World"
kind.split(" ")
print(kind.split(" "))    # ['Hello', 'World']
print(kind.split(" ")[0])    # Hello
print(kind)    # Hello World
```

- `A[i]=list(map(float,A[i])) `将其字符串转换为浮点数,返回列表

```python
A=[['1','2','3']]
A[i]=list(map(float,A[i]))
print(A)    # [[1.0, 2.0, 3.0]]
```
- `h=[abs(e-bestx) for e in x] `  #计算每个值与最优值之间的绝对差值,并存入列表h
- `print("输入矩阵为:\n{}".format(A))`  #打印矩阵A,{}作用是格式化输出,.format()方法用于格式化输出

- `ans.append([1-(lowx-x[i])/M])`    ans.append()函数是将列表元素添加到末尾,并返回None,所以ans.append()后面不能加括号
````python
ans=[]
for i in range(len(x)):
    ans.append([1-(lowx-x[i])/M])
print(ans)    # [[0.0], [0.0], [0.5], [0.5]]
````

-  `eval(input())`     输入字符串并执行,返回执行结果

````python
a=eval(input())
print(a)    # 输入字符串并执行,返回执行结果
````

- `X=v.reshape(-1,1)`    将向量v转换为矩阵,reshape(-1,1)表示将向量转换为1列矩阵

````python
v=[1,2,3]
X=v.reshape(-1,1)
print(X)    # [[1], [2], [3]]
````

- `X=np.hstack()`    合并矩阵,hstack()函数用于合并矩阵,返回合并后的矩阵

````python
A=[[1,2,3],[4,5,6]]
B=[[7,8,9],[10,11,12]]
X=np.hstack((A,B))
print(X)    # [[ 1  2  3  7  8  9]
            #  [ 4  5  6 10 11 12]]
````

- `X=X.astype('float') `    将矩阵X中的元素转换为浮点数

````python   
X=X.astype('float')
print(X)    # [[ 1.  2.  3.  7.  8.  9.]
            #  [ 4.  5.  6. 10. 11. 12.]]
````


- `X[:,j]**2`    计算矩阵X的每一列元素的平方

````python
X=np.array([[1,2,3],[4,5,6]])
for j in range(X.shape[1]):
    print(X[:,j]**2)    # [ 1  4 25]    为什么是**2呢,因为平方运算符是**
                        # [16 25 36]
````

- `np.square(X)`    计算矩阵X的每一元素的平方

````python
X=np.array([[1,2,3],[4,5,6]])
print(np.square(X))    # [[ 1  4 25]
                        #  [16 25 36]]
````

- `np.tile()`    重复矩阵(上下重复)

````python
X=np.array([[1,2,3],[4,5,6]])
Y=np.tile(X,(2,1))
print(Y)    # [[1 2 3]
            #  [4 5 6]
            #  [1 2 3]
            #  [4 5 6]]
````

- `print(f"第{i+1}个标准化后百分制得分为:{Score[i]}")`    

````python
for i in range(len(Score)):
    print(f"第{i+1}个标准化后百分制得分为:{Score[i]}")  #f""表示格式化字符串,{}表示占位符,i+1表示索引从1开始
````

In [6]:
a=eval(input())
print(a)

1234
